# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../output_data/cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,61.66,53,1,12.06,AU,1632189986
1,chuy,-33.6971,-53.4616,54.43,88,84,16.78,UY,1632189986
2,qrendi,35.8347,14.4583,76.66,88,0,9.22,MT,1632189987
3,kruisfontein,-34.0033,24.7314,53.02,74,83,15.73,ZA,1632189938
4,makakilo city,21.3524,-158.0865,83.55,49,1,9.22,US,1632189987


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the lat and lng as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

#Use the humidity as weight
humidity = weather_df["Humidity"].astype(float)


# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 5)

fig.add_layer(heat_layer)

#show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
# Cleave through the DataFrame to find ideal weather condition.
new_weather_df = weather_df

# A max temperature lower than 80 degrees & higher than 69.
new_weather_df = new_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (new_weather_df["Max Temp"] > 69)]

# Wind speed less than 10 mph
new_weather_df = new_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness
new_weather_df = new_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop rows with null values
new_weather_df = new_weather_df.dropna()

new_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,qrendi,35.8347,14.4583,76.66,88,0,9.22,MT,1632189987
18,tongchuan,35.0806,109.0897,76.53,33,0,4.16,CN,1632189993
77,kharan,28.5833,65.4167,75.92,16,0,3.98,PK,1632190014
121,awbari,26.5921,12.7805,79.56,20,0,7.43,LY,1632190030
191,shiyan,32.6475,110.7781,77.81,60,0,6.62,CN,1632190055
201,darnah,32.7670,22.6367,79.57,47,0,4.81,LY,1632190059
216,micheweni,-4.9667,39.8333,74.28,85,0,9.40,TZ,1632190064
269,kolondieba,11.0882,-6.8926,75.18,79,0,3.22,ML,1632190085
285,cayenne,4.9333,-52.3333,77.04,94,0,0.00,GF,1632189900
350,peabiru,-23.9128,-52.3431,78.33,29,0,2.37,BR,1632190119


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df = new_weather_df

# params dict for each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # Saving lat & lng rows from df into variables
    lat = row["Lat"]
    lng = row["Lng"]

    # Params to hold lat & lng location for each city
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Url + params to get json
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # Saving results from json response
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 2: qrendi.
Closest hotel is M'Anglu.
------------
Retrieving Results for Index 18: tongchuan.
Closest hotel is Tongchuan Hotel.
------------
Retrieving Results for Index 77: kharan.
Closest hotel is Dil Aram Bangla.
------------
Retrieving Results for Index 121: awbari.
Closest hotel is فندق أوباري.
------------
Retrieving Results for Index 191: shiyan.
Closest hotel is Fenghuang Garden Holiday Hotel.
------------
Retrieving Results for Index 201: darnah.
Closest hotel is فندق الفردوس.
------------
Retrieving Results for Index 216: micheweni.
Missing field/result... skipping.
------------
Retrieving Results for Index 269: kolondieba.
Closest hotel is Hotel Dakan.
------------
Retrieving Results for Index 285: cayenne.
Closest hotel is Hôtel Le Dronmi.
------------
Retrieving Results for Index 350: peabiru.
Closest hotel is La visa hotel.
------------
Retrieving Results for Index 395: pafos.
Closest hotel is Elysium Hotel.
------------
Retrieving Results for

In [52]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,qrendi,35.8347,14.4583,76.66,88,0,9.22,MT,1632189987,M'Anglu
18,tongchuan,35.0806,109.0897,76.53,33,0,4.16,CN,1632189993,Tongchuan Hotel
77,kharan,28.5833,65.4167,75.92,16,0,3.98,PK,1632190014,Dil Aram Bangla
121,awbari,26.5921,12.7805,79.56,20,0,7.43,LY,1632190030,فندق أوباري
191,shiyan,32.6475,110.7781,77.81,60,0,6.62,CN,1632190055,Fenghuang Garden Holiday Hotel
201,darnah,32.7670,22.6367,79.57,47,0,4.81,LY,1632190059,فندق الفردوس
216,micheweni,-4.9667,39.8333,74.28,85,0,9.40,TZ,1632190064,NaN
269,kolondieba,11.0882,-6.8926,75.18,79,0,3.22,ML,1632190085,Hotel Dakan
285,cayenne,4.9333,-52.3333,77.04,94,0,0.00,GF,1632189900,Hôtel Le Dronmi
350,peabiru,-23.9128,-52.3431,78.33,29,0,2.37,BR,1632190119,La visa hotel


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 155, 0, 0.5)',
    stroke_color='rgba(0, 0, 155, 0.5)', scale=5,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))